In [1]:
from tqdm import tqdm
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings(action='ignore')
pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)
pd.options.display.float_format="{:.5f}".format
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [2]:
data_prf = pd.read_csv('data/prf_dataset.csv')
data_reserve = pd.read_csv('data/new_data_reserve.csv')
data_cluster = pd.read_csv('data/new_data_cluster.csv')

In [3]:
data_reserve

,공연일자,공연시작시간,선예매여부,예매시작일,장르,휴게시간,공연명,내한여부,러닝타임,가격종류,R석,A석,요일,결제내역수,티켓판매량,전체좌석,일반예매율,멤버십합계,가중예매율,멤버십유입률,월
0,2023-10-03,1700,1,20230719,클래식,20,안드라스 쉬프 피아노 리사이틀,1,120,4,160000,60000,화요일,0,0,0,0.51596,0,0,0.14879,10
1,2023-10-19,1930,1,20230823,클래식,0,예술의전당 전관 개관 30주년 특별음악회-코리안챔버오케스트라 초청 콘서트,0,120,4,80000,20000,목요일,0,0,0,0.51146,0,0,0.12985,10
2,2023-11-04,1700,0,20230901,독주,20,바리톤 김기훈 리사이틀,0,90,3,100000,60000,토요일,0,0,0,0.47639,0,0,0.22579,11
3,2023-11-12,1700,1,20230906,교향곡,20,키릴 페트렌코 & 베를린 필하모닉 (11.12),1,120,5,550000,100000,일요일,0,0,0,0.49723,0,0,0.18600,11
4,2023-12-01,2000,1,20221210,교향곡,15,2023 서울시향 얍 판 츠베덴의 베토벤 삼중 협주곡 ②,1,100,5,100000,10000,금요일,0,0,0,0.45764,0,0,0.26033,12


In [4]:
data_cluster

,공연일자,공연시작시간,선예매여부,예매시작일,장르,휴게시간,공연명,내한여부,러닝타임,가격종류,R석,A석,요일,결제내역수,티켓판매량,전체좌석,일반예매율,멤버십합계,가중예매율,멤버십유입률,월,cluster,cluster_R석,cluster_A석
0,2023-10-03,1700,1,20230719,클래식,20,안드라스 쉬프 피아노 리사이틀,1,2시간,4,160000,60000,1,0,0,0,0,0,0,0,10,1,81696.96970,25454.54545
1,2023-10-19,1930,1,20230823,클래식,0,예술의전당 전관 개관 30주년 특별음악회-코리안챔버오케스트라 초청 콘서트,0,2시간,4,80000,20000,3,0,0,0,0,0,0,0,10,1,81696.96970,25454.54545
2,2023-11-04,1700,0,20230901,독주,20,바리톤 김기훈 리사이틀,0,1시간 30분,3,100000,60000,5,0,0,0,0,0,0,0,11,2,89978.72340,26936.17021
3,2023-11-12,1700,1,20230906,교향곡,20,키릴 페트렌코 & 베를린 필하모닉 (11.12),1,2시간,5,550000,100000,6,0,0,0,0,0,0,0,11,5,150553.84615,43876.92308
4,2023-12-01,2000,1,20221210,교향곡,15,2023 서울시향 얍 판 츠베덴의 베토벤 삼중 협주곡 ②,1,1시간 40분,5,100000,10000,4,0,0,0,0,0,0,0,12,5,150553.84615,43876.92308


In [5]:
data = data_reserve.copy()
data['cluster_R석'] = data_cluster['cluster_R석']
data['cluster_A석'] = data_cluster['cluster_A석']

In [6]:
data

,공연일자,공연시작시간,선예매여부,예매시작일,장르,휴게시간,공연명,내한여부,러닝타임,가격종류,R석,A석,요일,결제내역수,티켓판매량,전체좌석,일반예매율,멤버십합계,가중예매율,멤버십유입률,월,cluster_R석,cluster_A석
0,2023-10-03,1700,1,20230719,클래식,20,안드라스 쉬프 피아노 리사이틀,1,120,4,160000,60000,화요일,0,0,0,0.51596,0,0,0.14879,10,81696.96970,25454.54545
1,2023-10-19,1930,1,20230823,클래식,0,예술의전당 전관 개관 30주년 특별음악회-코리안챔버오케스트라 초청 콘서트,0,120,4,80000,20000,목요일,0,0,0,0.51146,0,0,0.12985,10,81696.96970,25454.54545
2,2023-11-04,1700,0,20230901,독주,20,바리톤 김기훈 리사이틀,0,90,3,100000,60000,토요일,0,0,0,0.47639,0,0,0.22579,11,89978.72340,26936.17021
3,2023-11-12,1700,1,20230906,교향곡,20,키릴 페트렌코 & 베를린 필하모닉 (11.12),1,120,5,550000,100000,일요일,0,0,0,0.49723,0,0,0.18600,11,150553.84615,43876.92308
4,2023-12-01,2000,1,20221210,교향곡,15,2023 서울시향 얍 판 츠베덴의 베토벤 삼중 협주곡 ②,1,100,5,100000,10000,금요일,0,0,0,0.45764,0,0,0.26033,12,150553.84615,43876.92308


In [7]:
import math

In [8]:
math.exp(0)

1.0

In [9]:
E_M = data_prf['멤버십유입률'].mean()

In [10]:
E_R = data_prf['일반예매율'].mean()

In [11]:
# '제안가격_최고가'와 '제안가격_최저가' 계산
data['제안가격_최고가'] = data['cluster_R석'] * (np.exp(data['멤버십유입률'])- math.exp(E_M) + 1) * (np.exp(data['일반예매율']) - math.exp(E_R) + 1)
data['제안가격_최저가'] = data['cluster_A석'] * (np.exp(data['멤버십유입률'])- math.exp(E_M) + 1) * (np.exp(data['일반예매율']) - math.exp(E_R) + 1)

In [12]:
data

,공연일자,공연시작시간,선예매여부,예매시작일,장르,휴게시간,공연명,내한여부,러닝타임,가격종류,R석,A석,요일,결제내역수,티켓판매량,전체좌석,일반예매율,멤버십합계,가중예매율,멤버십유입률,월,cluster_R석,cluster_A석,제안가격_최고가,제안가격_최저가
0,2023-10-03,1700,1,20230719,클래식,20,안드라스 쉬프 피아노 리사이틀,1,120,4,160000,60000,화요일,0,0,0,0.51596,0,0,0.14879,10,81696.96970,25454.54545,83362.26710,25973.40666
1,2023-10-19,1930,1,20230823,클래식,0,예술의전당 전관 개관 30주년 특별음악회-코리안챔버오케스트라 초청 콘서트,0,120,4,80000,20000,목요일,0,0,0,0.51146,0,0,0.12985,10,81696.96970,25454.54545,80940.83014,25218.95301
2,2023-11-04,1700,0,20230901,독주,20,바리톤 김기훈 리사이틀,0,90,3,100000,60000,토요일,0,0,0,0.47639,0,0,0.22579,11,89978.72340,26936.17021,94016.35509,28144.88190
3,2023-11-12,1700,1,20230906,교향곡,20,키릴 페트렌코 & 베를린 필하모닉 (11.12),1,120,5,550000,100000,일요일,0,0,0,0.49723,0,0,0.18600,11,150553.84615,43876.92308,155548.49650,45332.54772
4,2023-12-01,2000,1,20221210,교향곡,15,2023 서울시향 얍 판 츠베덴의 베토벤 삼중 협주곡 ②,1,100,5,100000,10000,금요일,0,0,0,0.45764,0,0,0.26033,12,150553.84615,43876.92308,158579.28020,46215.82946


In [13]:
result = data.drop(['결제내역수','티켓판매량','전체좌석','멤버십합계','가중예매율'], axis = 1)

In [14]:
result

,공연일자,공연시작시간,선예매여부,예매시작일,장르,휴게시간,공연명,내한여부,러닝타임,가격종류,R석,A석,요일,일반예매율,멤버십유입률,월,cluster_R석,cluster_A석,제안가격_최고가,제안가격_최저가
0,2023-10-03,1700,1,20230719,클래식,20,안드라스 쉬프 피아노 리사이틀,1,120,4,160000,60000,화요일,0.51596,0.14879,10,81696.96970,25454.54545,83362.26710,25973.40666
1,2023-10-19,1930,1,20230823,클래식,0,예술의전당 전관 개관 30주년 특별음악회-코리안챔버오케스트라 초청 콘서트,0,120,4,80000,20000,목요일,0.51146,0.12985,10,81696.96970,25454.54545,80940.83014,25218.95301
2,2023-11-04,1700,0,20230901,독주,20,바리톤 김기훈 리사이틀,0,90,3,100000,60000,토요일,0.47639,0.22579,11,89978.72340,26936.17021,94016.35509,28144.88190
3,2023-11-12,1700,1,20230906,교향곡,20,키릴 페트렌코 & 베를린 필하모닉 (11.12),1,120,5,550000,100000,일요일,0.49723,0.18600,11,150553.84615,43876.92308,155548.49650,45332.54772
4,2023-12-01,2000,1,20221210,교향곡,15,2023 서울시향 얍 판 츠베덴의 베토벤 삼중 협주곡 ②,1,100,5,100000,10000,금요일,0.45764,0.26033,12,150553.84615,43876.92308,158579.28020,46215.82946


In [15]:
result.to_csv('data/final_result.csv', index=False) 